In [ ]:
import os
import subprocess
from pathlib import Path

def run_atomsk(
    input_command: str,
    orientation: str,
    output_filename: str,
    atomsk_path: str,
    dft_workflow_root: Path,
    subfolder: str = "oriented_cells",
    duplication: str = "1 1 10"
) -> Path:
    """
    Run Atomsk to generate a structure with given parameters and save it to a subfolder under DFT_Workflow.

    Parameters
    ----------
    input_command : str
        Atomsk input command string (e.g., "fcc 3.6 Cu").
    orientation : str
        Orientation string for Atomsk (e.g., "[100] [010] [001]").
    output_filename : str
        The output file name (e.g., "Cu_oriented.lmp").
    atomsk_path : str
        Full path to the Atomsk binary (provided externally).
    dft_workflow_root : Path
        The root path to your DFT_Workflow directory.
    subfolder : str
        Name of the subdirectory inside DFT_Workflow to store output (default: "oriented_cells").
    duplication : str
        Duplication factor string (default: "1 1 10").

    Returns
    -------
    Path
        Full path to the generated structure file.
    """
    # Resolve output directory inside DFT_Workflow
    output_dir = dft_workflow_root / subfolder
    output_dir.mkdir(parents=True, exist_ok=True)

    # Build command
    create_cmd = f"{atomsk_path} --create {input_command} orient {orientation} -duplicate {duplication} {output_filename}"
    move_cmd = f"mv {output_filename} {output_dir}"

    try:
        subprocess.run(create_cmd, shell=True, check=True)
        subprocess.run(move_cmd, shell=True, check=True)
        print(f"Atomsk ran successfully. File saved to: {output_dir / output_filename}")
    except subprocess.CalledProcessError as e:
        print(f"Atomsk failed: {e}")
        return None

    return output_dir / output_filename


In [ ]:
from ase import io, Atoms
from ase.data import atomic_numbers, covalent_radii
from scipy.interpolate import griddata
from ovito.data import Container
import numpy as np
import os
from pathlib import Path
from typing import Union

def gamma_struct(
    structure: Union[str, Atoms],
    struct_format: str = 'vasp',
    interlayer_shift: float = 0.0,
    increments: int = 10,
    duplicate_z: int = 10,
    shift_plane: float = 0.5,
    write_file: bool = False,
    write_dir: Union[str, Path] = ".",
) -> list[tuple[float, float, float]]:
    """
    Generalized Gamma surface POSCAR generator

    Parameters
    ----------
    structure : str or ASE Atoms
        CIF file path or ASE Atoms object.
    interlayer_shift : float
        Additional z-coordinate shift for the sliding layer.
    increments : float
        Number of increments along x and y displacement (default: 10).
    duplicate_z : int
        Number of repetitions along z.
    shift_plane : float
        Fractional height (0–1) of the plane to shift atoms above.
    write_file : bool
        If True, writes displaced structures to file.
    write_dir : str or Path
        Directory to save files if write_file is True.

    Returns
    -------
    list of (x, y, energy)
        Interpolated GSFE surface data points.
    """

    # Load structure
    atoms = io.read(structure,format = struct_format) if isinstance(structure, str) else structure.copy()
    atoms = atoms.repeat((1, 1, duplicate_z))
    original_positions = atoms.get_positions()
    lattice_vectors = atoms.cell

    limits = atoms.cell.lengths()
    z_mid = limits[2] * shift_plane + interlayer_shift
    areaxy = np.linalg.norm(np.cross(lattice_vectors[0] , lattice_vectors[1]))    

    # Displacement ranges
    x_range = np.linspace(0,1,increments)
    y_range = np.linspace(0,1,increments)

    for y in y_range:
        for x in x_range:
            atoms.set_positions(original_positions)
            burger = lattice_vectors[0] * x + lattice_vectors[1] * y
            for atom in atoms:
                if atom.position[2] > z_mid:
                    atom.position += burger
            atoms.wrap()

            if write_file:
                out_file = Path(write_dir) / f"{Path(str(structure)).stem}_x_{round(x, 2)}_y_{round(y, 2)}.vasp"
                io.write(out_file, atoms)


In [ ]:
from ase import io, Atoms
from ase.data import atomic_numbers, covalent_radii
from scipy.interpolate import griddata
from ovito.data import Container
import numpy as np
import os
from pathlib import Path
from typing import Union

def gsfe_struct(
    structure: Union[str, Atoms],
    struct_format: str = 'vasp',
    interlayer_shift: float = 0.0,
    increments: int = 10,
    xmax : float = 1.0,
    ymax : float = 0.0,
    duplicate_z: int = 10,
    shift_plane: float = 0.5,
    write_file: bool = False,
    write_dir: Union[str, Path] = ".",
) -> list[tuple[float, float, float]]:
    """
    Generalized GSFE POSCAR generator

    Parameters
    ----------
    structure : str or ASE Atoms
        CIF file path or ASE Atoms object.
    interlayer_shift : float
        Additional z-coordinate shift for the sliding layer.
    increments : float
        Number of increments along x and y displacement (default: 10).
    xmax : float
        The final displacement length along the x direction (default: 1.0 Angstrom)
    ymax : float
        The final displacement length along the y direction (default: 0.0 Angstrom)
    duplicate_z : int
        Number of repetitions along z.
    shift_plane : float
        Fractional height (0–1) of the plane to shift atoms above.
    write_file : bool
        If True, writes displaced structures to file.
    write_dir : str or Path
        Directory to save files if write_file is True.

    Returns
    -------
    list of (x, y, energy)
        Interpolated GSFE surface data points.
    """

    # Load structure
    atoms = io.read(structure,format = struct_format) if isinstance(structure, str) else structure.copy()
    atoms = atoms.repeat((1, 1, duplicate_z))
    original_positions = atoms.get_positions()
    lattice_vectors = atoms.cell

    limits = atoms.cell.lengths()
    z_mid = limits[2] * shift_plane + interlayer_shift
    areaxy = np.linalg.norm(np.cross(lattice_vectors[0] , lattice_vectors[1]))    

    # Displacement ranges
    x_range = np.linspace(0,xmax/np.linalg.norm(lattice_vectors[0]),increments)
    y_range = np.linspace(0,ymax/np.linalg.norm(lattice_vectors[1]),increments)


    for x, y in zip(x_range,y_range):
        atoms.set_positions(original_positions)
        burger = lattice_vectors[0] * x + lattice_vectors[1] * y
        for atom in atoms:
            if atom.position[2] > z_mid:
                atom.position += burger
        atoms.wrap()

        if write_file:
            out_file = Path(write_dir) / f"{Path(str(structure)).stem}_x_{round(x, 2)}_y_{round(y, 2)}.vasp"
            io.write(out_file, atoms)
